In [57]:
import pandas as pd
import os
import re
from geopy import distance
#https://geopy.readthedocs.io/en/stable/#module-geopy.distance


In [58]:
airbnb_df = pd.read_csv('./Raw_Data/london_listings.csv', low_memory=False)
starbucks_df = pd.read_csv('./Raw_Data/starbucks.csv', low_memory=False)
reviews_df = pd.read_csv('./Raw_Data/london_reviews.csv', low_memory=False)

In [59]:
starbucks_df.loc[starbucks_df['countryCode']=='GB']

,Unnamed: 0,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
22721,16069,9155-152277,GB,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 10:...",membury-south-welcome-break-m-4-motorway-westb...,51.483556,-1.557143,"M4 Motorway Westbound, Junction 14/15",Lambourne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22722,16070,22194-218828,GB,LS,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",membury-south-welcome-break-dt-m-4-motorway-j-...,51.482387,-1.555109,M4 Motorway J14 15 membury,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22723,16071,18362-190424,GB,LS,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",membury-north-welcome-break-dt-m-4-motorway-ju...,51.481264,-1.556526,"M4 Motorway, Junction 14/15",Lambourne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22724,16072,9136-152279,GB,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 10:...",membury-north-welcome-break-m-4-motorway-eastb...,51.481177,-1.557422,M4 Motorway Eastbound Junction 14 15,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22725,16073,47832-260044,GB,FR,"[{'dayName': 'Today', 'hours': '10:00 AM to 8:...",swindon-great-western-outlet-unit-51-swindon-e...,51.562617,-1.798111,Unit 51,Great Western Designer Village,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23691,27459,21472-213065,GB,LS,"[{'dayName': 'Today', 'hours': '7:00 AM to 7:0...",rhyl-kinmel-park-eastbound-kinmel-park-a-55-ea...,53.270439,-3.519619,"Kinmel Park, A55 Eastbound",Bodelwyddon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23692,27859,47867-260072,GB,FR,"[{'dayName': 'Today', 'hours': '6:30 AM to 5:3...",welwyn-garden-city-howard-centre-unit-56-welwy...,51.802087,-0.204220,Unit 56,The Howard Centre,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23693,27860,9139-152280,GB,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 10:...",birchanger-welcome-break-m-11-motorway-junctio...,51.871540,0.194940,"M11 Motorway, Junction 8","Old Dunmow Road, Bishop Stortford",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23694,27861,27642-248279,GB,FR,"[{'dayName': 'Today', 'hours': '7:00 AM to 6:0...",bishops-stortford-jacksons-square-jacksons-squ...,51.870230,0.160930,Jacksons Square,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# AirBNB Listing Cleaning

In [60]:
display(airbnb_df.info())
display(airbnb_df.shape)
airbnb_columns = airbnb_df.columns.to_list()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87946 entries, 0 to 87945
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            87946 non-null  int64  
 1   listing_url                                   87946 non-null  object 
 2   scrape_id                                     87946 non-null  int64  
 3   last_scraped                                  87946 non-null  object 
 4   source                                        87946 non-null  object 
 5   name                                          87946 non-null  object 
 6   description                                   86679 non-null  object 
 7   neighborhood_overview                         47190 non-null  object 
 8   picture_url                                   87943 non-null  object 
 9   host_id                                       87946 non-null 

None

(87946, 75)

In [61]:
airbnb_columns_tokeep = \
['id',
 'description',
 'neighborhood_overview',
 'host_neighbourhood',
 'neighbourhood_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'price',
 'has_availability',
 'number_of_reviews',
 'number_of_reviews_ltm',
 'number_of_reviews_l30d',
 'first_review',
 'last_review',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'instant_bookable',
 'calculated_host_listings_count',
 'reviews_per_month']


In [62]:
#Reviewed total of 75 original columns, eliminated unnecessary columns or columns 
#with excessively null data
airbnb_df = airbnb_df[airbnb_columns_tokeep]
# filled null values with -1 for easy identification - no organic data would contain -1
airbnb_df.fillna(-1,inplace=True)
# airbnb_df

In [63]:
display(airbnb_df.info())
# reviewed for columns with inappropriate data types, identified:
# bathrooms_text, price, first_review, last_review

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87946 entries, 0 to 87945
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              87946 non-null  int64  
 1   description                     87946 non-null  object 
 2   neighborhood_overview           87946 non-null  object 
 3   host_neighbourhood              87946 non-null  object 
 4   neighbourhood_cleansed          87946 non-null  object 
 5   latitude                        87946 non-null  float64
 6   longitude                       87946 non-null  float64
 7   property_type                   87946 non-null  object 
 8   room_type                       87946 non-null  object 
 9   accommodates                    87946 non-null  int64  
 10  bathrooms_text                  87946 non-null  object 
 11  bedrooms                        87946 non-null  float64
 12  beds                            

None

In [64]:
#using regular expressions to extract the number of bathrooms
# bathroom_re_format = r'(.*?)( .*)'
# def extract_bathroom_count(string):
#     string = str(string)
#     re_return = re.match(bathroom_re_format, string)
#     return float(re_return.group(1)) if re_return else None
# airbnb_df['bathroom_count'] = airbnb_df['bathrooms_text'].apply(extract_bathroom_count)

#Using string methods to convert price to a numerical value
airbnb_df['price'] = airbnb_df['price'].str.replace(',','').str.strip('$')
airbnb_df['price'] = pd.to_numeric(airbnb_df['price'])

In [65]:
#Removing Price outliers (found to be likely glitches / faulty data)
airbnb_df = airbnb_df.loc[airbnb_df['price'] < 2001]
#Changing first and last reviews to datetimes
airbnb_df['first_review'] = pd.to_datetime(airbnb_df['first_review'], errors='coerce')
airbnb_df['last_review'] = pd.to_datetime(airbnb_df['last_review'], errors='coerce')

In [66]:
airbnb_df.sort_values(by='id',inplace=True)
airbnb_df.reset_index(drop=True, inplace=True)
airbnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87701 entries, 0 to 87700
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              87701 non-null  int64         
 1   description                     87701 non-null  object        
 2   neighborhood_overview           87701 non-null  object        
 3   host_neighbourhood              87701 non-null  object        
 4   neighbourhood_cleansed          87701 non-null  object        
 5   latitude                        87701 non-null  float64       
 6   longitude                       87701 non-null  float64       
 7   property_type                   87701 non-null  object        
 8   room_type                       87701 non-null  object        
 9   accommodates                    87701 non-null  int64         
 10  bathrooms_text                  87701 non-null  object        
 11  be

In [67]:
airbnb_df

,id,description,neighborhood_overview,host_neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month
0,13913,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,LB of Islington,Islington,51.568610,-0.112700,Private room in rental unit,Private room,1,...,4.80,4.72,4.72,4.74,4.82,4.69,4.69,f,2,0.26
1,15400,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,Chelsea,Kensington and Chelsea,51.487800,-0.168130,Entire rental unit,Entire home/apt,2,...,4.80,4.85,4.88,4.88,4.83,4.93,4.74,f,1,0.56
2,17402,"You'll have a great time in this beautiful, cl...","Fitzrovia is a very desirable trendy, arty and...",Fitzrovia,Westminster,51.521950,-0.140940,Entire rental unit,Entire home/apt,6,...,4.76,4.83,4.71,4.71,4.71,4.88,4.60,f,9,0.36
3,24328,"Artist house, bright high ceiling rooms for bo...","- Battersea is a quiet family area, easy acces...",Battersea,Wandsworth,51.470720,-0.162660,Entire townhouse,Entire home/apt,3,...,4.90,4.89,4.91,4.90,4.93,4.59,4.65,f,1,0.62
4,25123,Big room with double bed/ clean sheets/ clean ...,Barnet is one of the largest boroughs in Londo...,LB of Barnet,Barnet,51.574380,-0.210810,Private room in home,Private room,2,...,4.78,4.70,4.94,4.91,4.89,4.45,4.74,t,3,0.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87696,973781286754517228,Enjoy a Experience unique look in the center o...,-1,-1,Westminster,51.514860,-0.135980,Entire rental unit,Entire home/apt,3,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,f,2,-1.00
87697,973801695874775338,La Casita is a unique one bedroom flat in Sout...,-1,-1,Southwark,51.459042,-0.055458,Entire rental unit,Entire home/apt,2,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,f,1,-1.00
87698,973811685656289740,-1,-1,-1,Merton,51.406100,-0.236126,Private room in home,Private room,7,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,f,1,-1.00
87699,973882998775927897,<b>The space</b><br />Looking for a spacious h...,-1,Quaid Block,Hounslow,51.450997,-0.444319,Entire home,Entire home/apt,10,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,t,170,-1.00


In [68]:
airbnb_longitude_mean = airbnb_df['longitude'].mean()
airbnb_latitude_mean = airbnb_df['latitude'].mean()
bnb_center_of_mass = (airbnb_latitude_mean,airbnb_longitude_mean)

# Airbnb Review Cleaning

In [69]:
display(reviews_df.head())
reviews_columns = reviews_df.columns.to_list()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,13913,80770,2010-08-18,177109,Michael,My girlfriend and I hadn't known Alina before ...
1,13913,367568,2011-07-11,19835707,Mathias,Alina was a really good host. The flat is clea...
2,13913,529579,2011-09-13,1110304,Kristin,Alina is an amazing host. She made me feel rig...
3,13913,595481,2011-10-03,1216358,Camilla,"Alina's place is so nice, the room is big and ..."
4,13913,612947,2011-10-09,490840,Jorik,"Nice location in Islington area, good for shor..."


In [70]:
#Sorting numerically by listing ID to consolidate reviews by listing
reviews_df.sort_values(by='listing_id',inplace=True)
#Select Relevant Columns
reviews_df = reviews_df[['listing_id','date','comments']]

In [71]:
reviews_df

,listing_id,date,comments
0,13913,2010-08-18,My girlfriend and I hadn't known Alina before ...
23,13913,2022-05-20,This is the most cosy and beautifully-decorate...
24,13913,2022-05-22,"Alina was very welcoming, friendly and helpfu..."
25,13913,2022-06-10,"An arty flat, clean and comfortable big room, ..."
26,13913,2022-06-18,"Alina is a great host, very friendly, relaxed,..."
...,...,...,...
1649185,1040770808160963521,2023-12-08,Very clean and tidy
1649186,1041221637785631813,2023-12-10,Great location for exploring central London. T...
1649187,1041280679284501283,2023-12-10,I had an outstanding experience staying at Nat...
1649188,1041561781096383883,2023-12-10,My stay was amazing I highly recommend this ap...


In [72]:
def count_mentions(string):
    string = str(string)
    string = string.lower()
    mentions = string.count('starbucks')
    return mentions

reviews_df['sb_mentions'] = reviews_df['comments'].apply(count_mentions)

In [73]:
aggregate_reviews = reviews_df.groupby('listing_id').sum().reset_index()
aggregate_reviews

,listing_id,date,comments,sb_mentions
0,13913,2010-08-182022-05-202022-05-222022-06-102022-0...,My girlfriend and I hadn't known Alina before ...,0
1,15400,2016-07-262016-10-192016-10-142016-10-102016-0...,I cannot recommend this flat highly enough. It...,0
2,17402,2018-10-152018-10-092018-09-182018-09-112018-0...,Fantastic location - really in the middle of t...,0
3,24328,2015-09-062015-06-272015-09-032015-08-282015-0...,This was my first Airbnb experience. Joe's ho...,0
4,25123,2015-12-192015-12-272015-12-302016-01-302016-0...,Grace was fantastic the house and room is frie...,0
...,...,...,...,...
67650,1040770808160963521,2023-12-08,Very clean and tidy,0
67651,1041221637785631813,2023-12-10,Great location for exploring central London. T...,0
67652,1041280679284501283,2023-12-10,I had an outstanding experience staying at Nat...,0
67653,1041561781096383883,2023-12-10,My stay was amazing I highly recommend this ap...,0


# Starbucks Data Cleaning

In [74]:
display(starbucks_df.head())
starbucks_columns = starbucks_df.columns.to_list()

,Unnamed: 0,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,7958,75988-107245,US,LS,"[{'dayName': 'Today', 'hours': '8:00 AM to 4:0...",anc-alaska-ticketing-door-3-5000-west-internat...,61.174006,-149.981584,5000 West International Airport,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,city,NaN
1,7959,75697-94324,US,LS,"[{'dayName': 'Today', 'hours': '3:30 AM to 12:...",anc-concourse-c-gate-c-1-5000-w-intl-airport-r...,61.173768,-149.982953,5000 W Intl Airport Rd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,London,187.0
2,7960,74430-51979,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",carrs-anchorage-1805-1650-w-northern-lights-bl...,61.194274,-149.915872,1650 W Northern Lights Blvd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,New York,187.0
3,7961,79664-81037,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-anchorage-1812-1725-abbot-rd-anchorage...,61.137251,-149.956031,1725 Abbot Rd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,Chicago,178.0
4,7962,25371-240771,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 8:0...",benson-minnesota-1515-w-29th-ave-anchorage-ak-...,61.194103,-149.912474,1515 W 29th Ave,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Las Vegas,165.0


In [75]:
starbucks_columns
# Initial removal of un-used columns (address, store number, etc.)
starbucks_columns_tokeep = \
['Unnamed: 0',
 'countryCode',
 'ownershipTypeCode',
 'latitude',
 'longitude',
 'city',
 'countrySubdivisionCode',
 'postalCode']
starbucks_df = starbucks_df[starbucks_columns_tokeep]
starbucks_df.rename(columns={'Unnamed: 0':'store_id'},inplace=True)

In [76]:
#Filter to US based Colorado stores (searching for denver suburbs possibly included therein)
starbucks_df = starbucks_df.loc[(starbucks_df['countryCode'] == 'GB')
                                & (starbucks_df['countrySubdivisionCode'] == 'ENG')]

In [77]:
#Truncating zip+4 postal codes to 5 digit codes
starbucks_df['postalCode'] = starbucks_df['postalCode'].str[:5]

In [78]:
#calculate the distance to denver from each starbucks, populate dataframe row
def distance_to_bnb_center(row):
    lat = row['latitude']
    long = row['longitude']
    radius = round(distance.distance((lat,long),bnb_center_of_mass).miles,2)
    return radius

starbucks_df['distance_to_den'] = starbucks_df.apply(distance_to_bnb_center, axis=1)

In [79]:
#eliminating starbucks outside a 25 mile radius from the airbnb center
starbucks_df = starbucks_df.loc[starbucks_df['distance_to_den'] < 25]
#re-index after filters
starbucks_df.reset_index(inplace=True)

In [80]:
starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   256 non-null    int64  
 1   store_id                256 non-null    int64  
 2   countryCode             256 non-null    object 
 3   ownershipTypeCode       256 non-null    object 
 4   latitude                256 non-null    float64
 5   longitude               256 non-null    float64
 6   city                    256 non-null    object 
 7   countrySubdivisionCode  256 non-null    object 
 8   postalCode              255 non-null    object 
 9   distance_to_den         256 non-null    float64
dtypes: float64(3), int64(2), object(5)
memory usage: 20.1+ KB


# Early Analytics

In [81]:
#Short check to find how many listings do not have at least one review
listings_ids = list(airbnb_df['id'].unique())
reviews_ids = list(reviews_df['listing_id'].unique())
print(len(listings_ids)-len(reviews_ids))

20046


In [82]:
#Given 303 starbucks locations and 5367 airbnb locations, calculate a distance from each airbnb to each starbucks
#create two list, store them in each dataframe 
def starbucks_distance_calc(airbnb):
    """ Receives: a row from an airbnb listings dataframe 
    returns: a dictionary with the distance to every starbucks"""
    lat = airbnb['latitude']
    long = airbnb['longitude']
    bnb_location = (lat, long)
    distance_list = []
    for index, row in starbucks_df.iterrows():
        sb_lat = row['latitude']
        sb_long = row['longitude']
        sb_location = (sb_lat, sb_long)
        sb_id = row['store_id']
        radius = round(distance.distance(bnb_location,sb_location).miles,2)
        distance_list.append(radius)
    return distance_list
#Create the list of distances to every starbucks, applied to the airbnb_df
airbnb_df['starbucks_distances'] = airbnb_df.apply(starbucks_distance_calc, axis=1)

In [83]:
#Airbnb Distance calc (opposite of starbucks above)
def airbnb_distance_calc(starbucks):
    """ Receives: a row from an airbnb listings dataframe 
    returns: a dictionary with the distance to every starbucks"""
    lat = starbucks['latitude']
    long = starbucks['longitude']
    starbucks_location = (lat, long)
    distance_list = []
    for index, row in airbnb_df.iterrows():
        bnb_lat = row['latitude']
        bnb_long = row['longitude']
        bnb_location = (bnb_lat, bnb_long)
        bnb_id = row['id']
        radius = round(distance.distance(bnb_location,starbucks_location).miles,2)
        distance_list.append(radius)
    return distance_list
#Create the list of distances to every starbucks, applied to the airbnb_df
starbucks_df['airbnb_distances'] = starbucks_df.apply(airbnb_distance_calc, axis=1)

In [84]:
def mentions_lookup(row):
    bnb_id = row['id']
    try:
        mentions = aggregate_reviews.loc[aggregate_reviews['listing_id']==bnb_id,'sb_mentions'].item()
    except:
        mentions = 0
    return mentions
airbnb_df['starbucks_review_mentions'] = airbnb_df.apply(mentions_lookup, axis=1)

In [85]:
airbnb_df.set_index('id', inplace=True)
starbucks_df.set_index('store_id', inplace=True)
airbnb_df.to_csv('./Clean_Data/London_AirBNB_Data.csv', index=True)
starbucks_df.to_csv('./Clean_Data/London_Starbucks_Data.csv', index=True)

In [86]:
import ast
test_read_sb = pd.read_csv('./Clean_Data/London_Starbucks_Data.csv')
test_read_sb['airbnb_distances'] = test_read_sb['airbnb_distances'].apply(ast.literal_eval)

In [87]:
test_read_sb

,store_id,index,countryCode,ownershipTypeCode,latitude,longitude,city,countrySubdivisionCode,postalCode,distance_to_den,airbnb_distances
0,17111,23030,GB,CO,51.609615,-0.643035,Beaconsfield,ENG,HP9 1,23.25,"[23.01, 22.13, 22.47, 22.83, 18.77, 25.39, 17...."
1,17115,23034,GB,LS,51.588587,-0.627931,Beaconsfield,ENG,HP9 2,22.24,"[22.24, 21.01, 21.49, 21.65, 17.99, 24.45, 15...."
2,17116,23035,GB,LS,51.587445,-0.627850,Beaconsfield,ENG,HP9 2,22.21,"[22.23, 20.98, 21.47, 21.62, 17.99, 24.44, 15...."
3,17951,23178,GB,FR,51.759200,-0.439316,Hemel Hempstead,ENG,HP2 4,21.85,"[19.26, 22.1, 20.83, 23.23, 16.12, 22.72, 20.9..."
4,17952,23179,GB,FR,51.747670,-0.473320,Hemel Hempstead,ENG,HP1 1,22.18,"[19.84, 22.25, 21.17, 23.35, 16.46, 23.23, 20...."
...,...,...,...,...,...,...,...,...,...,...,...
251,27004,23624,GB,CO,51.287245,-0.154998,Reigate and Banstead,ENG,RH1 3,15.41,"[19.54, 13.88, 16.24, 12.69, 20.0, 16.79, 14.2..."
252,27055,23675,GB,FR,51.237909,-0.208984,Uxbridge,ENG,RH2 7,19.10,"[23.24, 17.37, 19.86, 16.22, 23.26, 20.69, 16...."
253,27056,23676,GB,LS,51.163360,-0.175130,West Sussex,ENG,RH6 0,24.02,"[28.15, 22.43, 24.83, 21.25, 28.46, 25.36, 21...."
254,27859,23692,GB,FR,51.802087,-0.204220,Welwyn Garden City,ENG,AL8 6,20.49,"[16.61, 21.78, 19.56, 22.98, 15.74, 20.06, 23...."
